# Demo 2: Multi-Agent Incident Response with IPC

## Overview

This notebook demonstrates building a multi-agent incident response system where:
- **3 separate processes** share one SochDB instance via **IPC (Inter-Process Communication)**
- **Process A**: Collects metrics and writes to KV storage
- **Process B**: Indexes runbooks into vector collection
- **Process C**: Monitors metrics, retrieves runbooks, manages incident state

### What You'll Learn

1. How to run SochDB in **IPC mode** (Unix socket)
2. How **multiple processes** can share one database
3. How to use **namespaces** for data isolation
4. How **hybrid retrieval** combines vector + keyword search
5. How to implement **ACID state machines**
6. How **concurrent writes** work without conflicts

---

## Setup

### Prerequisites

```bash
pip install sochdb openai tiktoken
export OPENAI_API_KEY="your-api-key-here"
```

### Import Dependencies

In [ ]:
import os
import sys
import time
from pathlib import Path
from datetime import datetime

sys.path.insert(0, str(Path.cwd().parent))

from sochdb import IpcClient, ContextQuery, DeduplicationStrategy
from shared.llm_client import LLMClient
from shared.embeddings import EmbeddingClient

print("✅ All dependencies imported successfully!")

---

## Understanding IPC Mode

### 📚 Concept: IPC (Inter-Process Communication)

**Traditional approach**: Each process has its own database connection, leading to:
- Data duplication
- Synchronization issues
- Complex message passing

**SochDB IPC approach**: One database server, multiple client processes:
```
SochDB Server (Unix Socket)
    ↓
    ├── Process A (Metrics Collector)
    ├── Process B (Runbook Indexer)
    └── Process C (Incident Commander)
```

**Benefits**:
- Shared state automatically
- No manual synchronization
- ACID guarantees across processes

### How-To: Start SochDB Server

```bash
# In a terminal
sochdb-server --db ./incident_db
```

This creates a Unix socket at `./incident_db/sochdb.sock`

---

## Step 1: Connect to IPC Server

### How-To: Connect via IPC Client

In [ ]:
# Note: This assumes SochDB server is running
# If not started, run: sochdb-server --db ./incident_db

socket_path = "./incident_db/sochdb.sock"

try:
    client = IpcClient.connect(socket_path)
    print(f"✅ Connected to SochDB server via IPC")
    print(f"   Socket: {socket_path}")
except Exception as e:
    print(f"❌ Could not connect: {e}")
    print(f"\n💡 Make sure to start the server first:")
    print(f"   sochdb-server --db ./incident_db")

---

## Step 2: Simulate Process A - Metrics Collector

### 📚 Concept: KV Storage for Time-Series Metrics

Use KV paths to store metrics:
- `metrics/latest/*` - Current values
- `metrics/{metric_name}/{timestamp}` - Historical data

**Why this works**: Fast writes, key-based access, no schema needed.

### How-To: Write Metrics

In [ ]:
import random

def collect_metrics(client, iterations=5):
    """Simulate metrics collection."""
    print("📊 Simulating metrics collection...\n")
    
    for i in range(iterations):
        timestamp = datetime.now().isoformat()
        
        # Simulate increasing latency/errors (incident trigger)
        latency = random.randint(200, 800) if i < 3 else random.randint(1000, 2000)
        error_rate = round(random.uniform(0.5, 3.0), 2) if i < 3 else round(random.uniform(5.0, 10.0), 2)
        
        # Write to KV
        client.put(b"metrics/latest/latency_p99", str(latency).encode())
        client.put(b"metrics/latest/error_rate", str(error_rate).encode())
        client.put(b"metrics/latest/timestamp", timestamp.encode())
        
        print(f"   [{i+1}] Latency: {latency}ms | Error Rate: {error_rate}%")
        time.sleep(1)
    
    # Trigger incident
    if latency > 1000 and error_rate > 5.0:
        client.put(b"incidents/current/severity", b"HIGH")
        client.put(b"incidents/current/latency", str(latency).encode())
        client.put(b"incidents/current/error_rate", str(error_rate).encode())
        print(f"\n🚨 INCIDENT TRIGGERED! Latency: {latency}ms, Error: {error_rate}%")

# Run simulation
collect_metrics(client)

---

## Step 3: Simulate Process B - Runbook Indexer

### 📚 Concept: Namespace Isolation

Namespaces keep data separated:
```python
ns = client.namespace("incident_ops")
```

- Collections live in namespaces
- Prevents name collisions
- Logical separation (tenants, environments, etc.)

### How-To: Index Documents into Vector Collection

In [ ]:
embedding_client = EmbeddingClient()
dimension = embedding_client.dimension

# Sample runbook content
runbooks = [
    {
        "id": "latency_1",
        "text": "Latency spike: Check recent deployments. If spike coincides with deployment, consider rollback.",
        "source": "latency_spike.txt"
    },
    {
        "id": "latency_2",
        "text": "High latency often caused by database overload. Check slow query log and connection pool utilization.",
        "source": "latency_spike.txt"
    },
    {
        "id": "rollback_1",
        "text": "To rollback deployment: kubectl rollout undo deployment/app-name. Monitor metrics for 5 minutes after.",
        "source": "deployment_rollback.txt"
    }
]

# Create namespace and collection
ns = client.namespace("incident_ops")
collection = ns.create_collection("runbooks", dimension=dimension)

print(f"📚 Indexing runbooks into 'incident_ops' namespace...\n")

for runbook in runbooks:
    embedding = embedding_client.embed(runbook["text"])
    
    collection.add_document(
        id=runbook["id"],
        embedding=embedding,
        text=runbook["text"],
        metadata={"source": runbook["source"]}
    )
    print(f"   ✓ Indexed: {runbook['id']}")

print(f"\n✅ Indexed {len(runbooks)} runbooks in vector collection")

---

## Step 4: Simulate Process C - Incident Commander

### 📚 Concept: Hybrid Retrieval with RRF

**Hybrid Search** = Vector Search + Keyword Search

**RRF (Reciprocal Rank Fusion)** combines results:
1. Vector search finds semantically similar documents
2. Keyword search finds exact matches
3. RRF merges and ranks results

**Why this matters**: Better recall than vector-only or keyword-only search.

### How-To: Build Hybrid Query

In [ ]:
# Check incident status
severity = client.get(b"incidents/current/severity")
latency = client.get(b"incidents/current/latency")
error_rate = client.get(b"incidents/current/error_rate")

if severity and severity.decode() == "HIGH":
    print("🚨 INCIDENT DETECTED!")
    print(f"   Latency: {latency.decode()}ms")
    print(f"   Error Rate: {error_rate.decode()}%")
    print(f"\n🔍 Retrieving relevant runbooks...\n")
    
    # Build hybrid query
    query = f"high latency {latency.decode()}ms error rate {error_rate.decode()}%"
    query_embedding = embedding_client.embed(query)
    
    ns = client.namespace("incident_ops")
    collection = ns.collection("runbooks")
    
    ctx = (
        ContextQuery(collection)
        .add_vector_query(query_embedding, weight=0.6)  # Semantic similarity
        .add_keyword_query("latency deployment rollback", weight=0.4)  # Keyword match
        .with_token_budget(2000)
        .with_deduplication(DeduplicationStrategy.SEMANTIC)
        .execute()
    )
    
    print(f"📄 Retrieved {len(ctx.documents)} relevant runbooks:")
    for i, doc in enumerate(ctx.documents, 1):
        print(f"\n   {i}. {doc.text}")
        print(f"      Source: {doc.metadata.get('source', 'unknown')}")
else:
    print("✅ No incident detected. System healthy.")

---

## Step 5: Generate Mitigation Plan

In [ ]:
if severity and severity.decode() == "HIGH":
    llm = LLMClient()
    
    system_message = """You are an incident commander. Analyze metrics and runbook guidance.
Suggest concrete mitigation actions in priority order."""
    
    prompt = f"""Incident Details:
- Latency P99: {latency.decode()}ms (threshold: 1000ms)
- Error Rate: {error_rate.decode()}% (threshold: 5%)

Relevant Runbooks:
{ctx.as_markdown()}

Provide:
1. Most likely root cause
2. Immediate mitigation actions (priority order)
3. Next steps after mitigation
"""
    
    response = llm.complete(prompt, system_message=system_message)
    
    print("\n" + "="*70)
    print("MITIGATION PLAN")
    print("="*70)
    print(response)
    print("="*70)

---

## Step 6: Manage Incident State with ACID

### 📚 Concept: State Machine Transitions

Incident lifecycle:
```
NONE → OPEN → MITIGATING → RESOLVED
```

Each transition is atomic:
- Update state
- Record timestamp
- Log to history

**All or nothing** - guaranteed consistent.

### How-To: Update State Atomically

In [ ]:
def update_incident_state(client, state, details):
    """Update incident state with atomic writes."""
    timestamp = datetime.now().isoformat()
    
    # All 3 writes are atomic in IPC mode
    client.put(b"incidents/current/state", state.encode())
    client.put(b"incidents/current/last_update", timestamp.encode())
    client.put(f"incidents/history/{timestamp}".encode(), f"{state}: {details}".encode())
    
    print(f"📝 State transition → {state}")
    print(f"   Timestamp: {timestamp}")
    print(f"   Details: {details}")

# Transition through states
if severity and severity.decode() == "HIGH":
    print("\n" + "="*70)
    print("INCIDENT STATE MANAGEMENT")
    print("="*70 + "\n")
    
    update_incident_state(client, "OPEN", "Incident detected, analyzing...")
    time.sleep(1)
    
    update_incident_state(client, "MITIGATING", "Executing rollback procedure")
    time.sleep(1)
    
    update_incident_state(client, "RESOLVED", "Metrics returned to normal")
    
    # Clear incident
    client.put(b"incidents/current/severity", b"NONE")
    
    print("\n✅ Incident resolved!")

---

## Step 7: Verify State History

In [ ]:
print("📚 Incident History:")
print("="*70)

# In real implementation, you'd list keys with prefix
# For this demo, we'll check current state
current_state = client.get(b"incidents/current/state")
last_update = client.get(b"incidents/current/last_update")
severity = client.get(b"incidents/current/severity")

print(f"Current State: {current_state.decode() if current_state else 'NONE'}")
print(f"Last Update: {last_update.decode() if last_update else 'N/A'}")
print(f"Severity: {severity.decode() if severity else 'NONE'}")
print("="*70)

---

## Understanding Multi-Process Coordination

### 💡 Key Insight: Shared State Without Message Passing

In this demo, we simulated 3 processes in one notebook. In production:

**Process A (Collector)**:
```python
while True:
    metrics = get_metrics()
    client.put(b"metrics/latest/...", metrics)
    time.sleep(5)
```

**Process B (Indexer)**:
```python
for runbook in watch_directory():
    embedding = embed(runbook)
    collection.add_document(...)
```

**Process C (Commander)**:
```python
while True:
    metrics = client.get(b"metrics/latest/...")
    if is_incident(metrics):
        runbooks = query_collection(...)
        plan = generate_plan(runbooks)
        update_state("MITIGATING")
```

**No RabbitMQ, Kafka, or Redis Pub/Sub needed!**

All processes read/write the same SochDB instance. State is automatically shared.

---

## Summary: What We Accomplished

### ✅ Features Demonstrated

1. **IPC Mode** - Connected to SochDB server via Unix socket  
2. **Shared State** - Multiple "processes" accessing same data
3. **Namespaces** - Isolated `incident_ops` namespace
4. **Hybrid Retrieval** - Combined vector (0.6) + keyword (0.4) search
5. **Token Budgeting** - Retrieved runbooks under 2000 token limit
6. **State Machine** - ACID transitions (NONE → OPEN → MITIGATING → RESOLVED)

### 💡 Key Insights

**No Message Queue Needed**
- Traditional: Process A → Kafka → Process B → Redis → Process C
- SochDB: All processes read/write shared database

**Automatic Consistency**
- ACID guarantees across all processes
- No eventual consistency issues
- No manual synchronization

**Hybrid Search Works Better**
- Vector-only: Misses exact keyword matches
- Keyword-only: Misses semantic similarity
- Hybrid with RRF: Best of both worlds

### 🚀 Next Steps

Run the actual multi-process demo:
```bash
cd ../2_incident_response
./run_demo.sh
```

Watch 3 separate processes coordinate through shared SochDB.

Try modifying:
- Add more runbooks and see hybrid search adapt
- Change weights (vector vs keyword)
- Add more metrics to trigger different incidents
- Implement more complex state machines

---

## Resources

- [SochDB Documentation](https://github.com/sochdb/sochdb)
- [IPC Mode Guide](https://github.com/sochdb/sochdb-python-sdk)
- [Demo Source Code](../2_incident_response/)
